In [3]:
!pip install chromadb

In [4]:
import librosa
import numpy as np
from chromadb.config import Settings
from chromadb import Client

In [5]:
# Step 1: Function to extract audio embeddings using Mel-frequency cepstral coefficients (MFCC)
def extract_audio_embedding(audio_path):
    y, sr = librosa.load(audio_path, sr=None)  # Load audio file
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)  # Extract MFCC features
    embedding = np.mean(mfcc, axis=1)  # Take the mean across time frames
    return embedding

In [6]:
# Step 2: Initialize ChromaDB client
client = Client(Settings())
collection = client.get_or_create_collection(name="audio_collection")

In [10]:
# Step 3: Add audio files to the vector database
def add_audio_to_database(audio_paths):
    for idx, audio_path in enumerate(audio_paths):
        embedding = extract_audio_embedding(audio_path)
        metadata = {"path": audio_path}
        collection.add(
            ids=[f"audio_{idx}"],  # Changed 'embedding_ids' to 'ids'
            embeddings=[embedding],
            metadatas=[metadata]
        )


In [13]:
# Step 4: Perform similarity search for an audio query
def search_similar_audio(query_audio_path, top_k=3):
    query_embedding = extract_audio_embedding(query_audio_path)
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)
    # Accessing the list of metadatas correctly:
    for result_metadata in results["metadatas"]:  # Iterate through the list of metadatas
        print("Similar Audio Found:", result_metadata[0]["path"])  # Access the path from the first element

In [14]:
# Step 5: Example usage
if __name__ == "__main__":
    # Example audio files (replace with your actual audio file paths)
    audio_files = [
        "/content/sample_data/sample-9s.mp3",  # Replace with real audio file paths
        "/content/sample_data/sample-6s.mp3",
        "/content/sample_data/sample-3s.mp3"
    ]

    # Add audio files to the database
    add_audio_to_database(audio_files)

    # Query with a new audio file
    query_audio = "/content/sample_data/audio-file-sample.mp3"  # Replace with the query audio file path
    search_similar_audio(query_audio, top_k=3)


Similar Audio Found: /content/sample_data/sample-9s.mp3
